## __1. Carga de dados__

#### __Objtivos__

- Explorar os arquivos csv com dados metereológicos
- Adaptar os tipos de dados para melhor se encaixarem no projeto
- Criar um banco de dados (clima.db) para carga de dados no dashboard
- Criar funções para criação, conexão e consulta dos dados 
- Testar as funções criadas

#### __Descrição__
    
    
   - Os arquivos forão obtidos do INMET(Instituto Nacional de Meteorologia)
   - É possível fazer downloads de dados obtidos em diversas estações espalhadas pelo país
   - Foram escolhidas apenas os dados das cidades do estado da Paraíba disponíveis
   - Perírodo
       - 01/01/2008 - 31/12/2018

- Dados de medição 
    - Temperatura de Bulbo Seco
    - Temperautra de Bulbo Úmido
    - Umidade
    - Pressão Atmosférica
    - Direção do Vento
    - Velocidade do vento
    - Nebulosidade

- Dados de Informação
    - Latitude
    - Longitude
    - Altitude

- Nomes dos arquivos
    - Os dados estão salvos em formato csv
    - O nome do arquivo é composto pelo nome da cidade junto ao sufixo _clima, além da extensão
    - Para as cidades com nomes compostos, o espaço entre os nomes é ignorado
    
- Banco de dados
    - Arquivo clima.db criado com SQLite3
    - As funções serão criadas apenas com auxílio de dataframes (pandas) e com a própria api do SQLite3

#### Explorando arquivo csv: João Pessoa

In [1]:
# Carregando biblioteca
import pandas as pd

In [3]:
#Leitura do arquivo
#O arquivo csv tem colunas semparadas por ';' 
df_jp = pd.read_csv("../dados/JoaoPessoa_clima.csv", sep = ";")

In [4]:
#5 primeiras linhas
df_jp.head()

,Estacao,Data,Hora,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade,Unnamed: 10
0,82798,01/01/2008,0,26.8,22.6,68.0,1012.1,14.0,3.3,3.0,NaN
1,82798,01/01/2008,1200,29.4,23.3,58.0,1012.7,14.0,3.5,4.0,NaN
2,82798,01/01/2008,1800,29.6,23.5,59.0,1011.2,14.0,3.3,3.0,NaN
3,82798,02/01/2008,0,26.6,22.2,69.0,1012.8,14.0,2.6,4.0,NaN
4,82798,02/01/2008,1200,29.8,23.9,61.0,1013.3,14.0,3.3,3.0,NaN


- A coluna Estacao pode ser descartada
- A última coluna deve ser descartada.
- As colunas Data e Hora podem ser combinadas em uma só coluna do tipo datetime
- A coluna DireçãoVento, apesar de represntada por valores numéricos, é categórica
    - A informação dos labels pode ser decodificada com o arquivo __DirecaoVento.csv__

In [5]:
df_jp.dtypes

Estacao                int64
Data                  object
Hora                   int64
TempBulboSeco        float64
TempBulboUmido       float64
UmidadeRelativa      float64
PressaoAtmEstacao    float64
DirecaoVento         float64
VelocidadeVento      float64
Nebulosidade         float64
Unnamed: 10          float64
dtype: object

#### __Modificando estrutura dos dados__

1. Excluir coluna estação
2. Excluir a última coluna
3. Combinar as Data e Hora em uma única coluna chamada Horario
4. Modificar tipo de dados de Horario para datetime
5. Escolher Horario como índice da tabela
6. Utilizar dados da tabela DirecaoVento.csv para modificar valores da coluna DirecaoVento

In [18]:
#Mudando especificações na carga de dados
df_jp = pd.read_csv("../dados/JoaoPessoa_clima.csv",
            delimiter = ";",
            usecols=list(range(1,10)),               #Exclui a primeria(Estacao) e última coluna
            dtype={'Hora':'str','DirecaoVento':'str'},
            parse_dates=[['Data', 'Hora']],       #Escolhe a combinação das colunas como ti datetime
            dayfirst = True,
            index_col = 0
            )   

In [12]:
df_jp.head(10)

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Data_Hora,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,14,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,14,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,14,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,14,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,14,3.3,3.0
2008-01-02 18:00:00,29.6,24.1,63.0,1010.8,14,4.6,3.0
2008-01-03 00:00:00,27.0,23.9,77.0,1013.8,14,3.3,5.0
2008-01-03 12:00:00,29.8,24.3,64.0,1014.5,14,6.0,4.0
2008-01-03 18:00:00,29.8,24.7,66.0,1012.4,18,5.6,4.0


In [19]:
df_jp.dtypes

TempBulboSeco        float64
TempBulboUmido       float64
UmidadeRelativa      float64
PressaoAtmEstacao    float64
DirecaoVento          object
VelocidadeVento      float64
Nebulosidade         float64
dtype: object

In [20]:
#Modificando nome do índice
df_jp.index.rename('Horario', inplace=True)
df_jp.index

DatetimeIndex(['2008-01-01 00:00:00', '2008-01-01 12:00:00',
               '2008-01-01 18:00:00', '2008-01-02 00:00:00',
               '2008-01-02 12:00:00', '2008-01-02 18:00:00',
               '2008-01-03 00:00:00', '2008-01-03 12:00:00',
               '2008-01-03 18:00:00', '2008-01-04 00:00:00',
               ...
               '2018-12-28 18:00:00', '2018-12-29 00:00:00',
               '2018-12-29 12:00:00', '2018-12-29 18:00:00',
               '2018-12-30 00:00:00', '2018-12-30 12:00:00',
               '2018-12-30 18:00:00', '2018-12-31 00:00:00',
               '2018-12-31 12:00:00', '2018-12-31 18:00:00'],
              dtype='datetime64[ns]', name='Horario', length=11938, freq=None)

In [22]:
#Carregando arquivo com informações sobre a direção do vento
vento = pd.read_csv('../dados/DirecaoVento.csv', delimiter=';', dtype={'Codigo':'str' })
vento.head()
 

,Codigo,Descricao,Sigla
0,0,Calma,C
1,1,Norte - Nordeste,NNE
2,2,Norte - Nordeste,NNE
3,3,Norte - Nordeste,NNE
4,4,Norte - Nordeste,NNE


In [23]:
vento.dtypes

Codigo       object
Descricao    object
Sigla        object
dtype: object

In [27]:
#Criando dicionário para mapear código de direções do vento
mapper = dict(zip(vento['Codigo'], vento['Descricao']))
mapper

{'0': 'Calma',
 '1': 'Norte - Nordeste',
 '2': 'Norte - Nordeste',
 '3': 'Norte - Nordeste',
 '4': 'Norte - Nordeste',
 '5': 'Nordeste',
 '6': 'Nordeste',
 '7': 'Leste - Nordeste',
 '8': 'Leste - Nordeste',
 '9': 'Leste',
 '10': 'Leste',
 '11': 'Leste - Sudeste',
 '12': 'Leste - Sudeste',
 '13': 'Leste - Sudeste',
 '14': 'Sudeste',
 '15': 'Sudeste',
 '16': 'Sul - Sudeste',
 '17': 'Sul - Sudeste',
 '18': 'Sul',
 '19': 'Sul',
 '20': 'Sul - Sudoeste',
 '21': 'Sul - Sudoeste',
 '22': 'Sul - Sudoeste',
 '23': 'Sudoeste',
 '24': 'Sudoeste',
 '25': 'Oeste - Sudoeste',
 '26': 'Oeste - Sudoeste',
 '27': 'Oeste',
 '28': 'Oeste',
 '29': 'Oeste - Noroeste',
 '30': 'Oeste - Noroeste',
 '31': 'Oeste - Noroeste',
 '32': 'Noroeste',
 '33': 'Noroeste',
 '34': 'Norte - Noroeste',
 '35': 'Norte - Noroeste',
 '36': 'Norte'}

In [30]:
df_jp['DirecaoVento']=df_jp['DirecaoVento'].map(mapper)
df_jp.head()

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Horario,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.3,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.5,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.3,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.6,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.3,3.0


### __Função para carregar arquivos csv__

- A função deve receber um arquivo de entrada e retornar um dataframe
- As alterações feitas no datafram acima devem ser incluídas na função
- O dataframe de retorno deve estar com a estrutura pronta para ser salvo como uma tabela
- A função deve poder ler também o arquivo de informações Info.csv 
- Lembrando, todos os arquivos de medições metereológicas terminam com '_clima.csv'

In [34]:
def carrega_csv(arquivo):
    
    #arquivo: Arquivo do tipo csv
    #retorno: Dataframe pandas
    
    if "_clima" in arquivo:
        df = pd.read_csv("../dados/JoaoPessoa_clima.csv",
            delimiter = ";",
            usecols=list(range(1,10)),               #Exclui a primeria(Estacao) e última coluna
            dtype={'Hora':'str','DirecaoVento':'str'},
            parse_dates=[['Data', 'Hora']],       #Escolhe a combinação das colunas como ti datetime
            dayfirst = True,
            index_col = 0
        )   
        vento = pd.read_csv('../dados/DirecaoVento.csv', delimiter=';', dtype={'Codigo':'str' })
        mapper = dict(zip(vento['Codigo'], vento['Descricao']))
        df['DirecaoVento']=df['DirecaoVento'].map(mapper)

    else :
        try:
            df = pd.read_csv(arquivo,
                            sep=";" 
                            )
        except:
            raise Exception("Arquivo não existe")
                
    return df

In [35]:
#Testando função com dados info
carrega_csv("../dados/Info.csv")

,Cidade,Latitude,Longitude,Altitude
0,Areia,-6.97,-35.68,574.62
1,Campina-Grande,-7.22,-35.88,547.56
2,Joao-Pessoa,-7.10,-34.86,7.43
3,Monteiro,-7.88,-37.06,603.66
4,Patos,-7.01,-37.26,249.09
5,Sao-Goncalo,-6.75,-38.21,233.06


In [36]:
#Testando função com dados metereológicos
carrega_csv("../dados/JoaoPessoa_clima.csv")

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Data_Hora,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.30000,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.50000,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.30000,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.60000,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.30000,3.0
...,...,...,...,...,...,...,...
2018-12-30 12:00:00,30.8,NaN,57.0,1012.7,Sudeste,1.54332,5.0
2018-12-30 18:00:00,NaN,NaN,59.0,1010.3,Leste,3.08664,5.0
2018-12-31 00:00:00,27.8,NaN,79.0,1013.4,Sudeste,3.08664,5.0


In [37]:
carrega_csv("../dados/Areia_clima.csv")

,TempBulboSeco,TempBulboUmido,UmidadeRelativa,PressaoAtmEstacao,DirecaoVento,VelocidadeVento,Nebulosidade
Data_Hora,,,,,,,
2008-01-01 00:00:00,26.8,22.6,68.0,1012.1,Sudeste,3.30000,3.0
2008-01-01 12:00:00,29.4,23.3,58.0,1012.7,Sudeste,3.50000,4.0
2008-01-01 18:00:00,29.6,23.5,59.0,1011.2,Sudeste,3.30000,3.0
2008-01-02 00:00:00,26.6,22.2,69.0,1012.8,Sudeste,2.60000,4.0
2008-01-02 12:00:00,29.8,23.9,61.0,1013.3,Sudeste,3.30000,3.0
...,...,...,...,...,...,...,...
2018-12-30 12:00:00,30.8,NaN,57.0,1012.7,Sudeste,1.54332,5.0
2018-12-30 18:00:00,NaN,NaN,59.0,1010.3,Leste,3.08664,5.0
2018-12-31 00:00:00,27.8,NaN,79.0,1013.4,Sudeste,3.08664,5.0


A função carrega_csv pode ser encontrada no módulo de apoio __bdconfig.py__ 